In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
dataset.head(5)

In [ ]:
X=dataset.drop('class',axis=1)
y=dataset['class']

In [ ]:
dataset.info()

In [ ]:
X.describe(include='all').T

In [ ]:
column_names=list()
for names in X.columns:
    column_names.append(names)
    
column_names

In [ ]:
X=pd.get_dummies(data=X, columns=column_names,drop_first=True)

In [ ]:
X.info()

In [ ]:
X.head(5)

In [ ]:
y=pd.get_dummies(y,drop_first=True)
y.head(5)

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# Initialising the ANN
classifier = Sequential()


classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = 95))

classifier.add(Dense(units=128,kernel_initializer='uniform',activation='relu'))


classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

# Compiling the ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
early_stopping=tensorflow.keras.callbacks.EarlyStopping(patience=2)

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 80, epochs = 100,validation_data=(X_test,y_test),verbose=2,callbacks=[early_stopping])

In [ ]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,y_pred)
report=classification_report(y_test,y_pred)
print(report)

In [ ]:
#Roc curve
from sklearn.metrics import roc_curve,auc
fpr,tpr,ThreshHold=roc_curve(y_test,y_pred)
aucValue=auc(fpr,tpr)

import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
plt.plot(fpr,tpr,label='AUC %0.2f'% aucValue)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()